In [1]:
from utils import get_lexicon, chars_to_remove
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
lex = get_lexicon()

f = "sample.txt"
with open(f, "r") as file:
    sample = file.read()

def preprocess(s:str):
    for c in chars_to_remove:
        s = s.replace(c, "")
    s = s.casefold()
    return s

def count_variants(s: str, lex):
    counts = dict()
    for word in preprocess(s).split():
        variant = lex.get(word, None)
        if not variant:
            continue
        logging.info(f"Found word {word}, presumed variant: {variant}.")
        counts[variant] = counts.get(variant, 0) + 1
    return counts

In [2]:
count_variants(sample, lex)


2022-02-15 14:58:02,392 Found word tires, presumed variant: A.
2022-02-15 14:58:02,395 Found word airplanes, presumed variant: A.
2022-02-15 14:58:02,396 Found word traveler, presumed variant: A.
2022-02-15 14:58:02,397 Found word traveler, presumed variant: A.
2022-02-15 14:58:02,398 Found word traveler, presumed variant: A.
2022-02-15 14:58:02,400 Found word traveler, presumed variant: A.
2022-02-15 14:58:02,400 Found word traveler, presumed variant: A.
2022-02-15 14:58:02,401 Found word analyses, presumed variant: B.
2022-02-15 14:58:02,402 Found word et, presumed variant: A.
2022-02-15 14:58:02,403 Found word practice, presumed variant: A.
2022-02-15 14:58:02,403 Found word programs, presumed variant: A.
2022-02-15 14:58:02,404 Found word program, presumed variant: A.
2022-02-15 14:58:02,405 Found word stories, presumed variant: A.


{'A': 12, 'B': 1}